In [ ]:
import os
import sys
import wandb
from huggingface_hub import login

In [ ]:
#sys.path.append("../")

In [ ]:
wandb.login()

In [ ]:
login()

In [34]:
!accelerate launch --mixed_precision="fp16" /w/340/vishnouvina/mobilediffusion/lora_distill_training.py \
    --pretrained_model_name_or_path="SG161222/Realistic_Vision_V4.0" \
    --dataset_name="fantasyfish/laion-art" \
    --caption_column="text"\
    --resolution=512 --center_crop --random_flip \
    --num_train_epochs=5 \
    --gradient_accumulation_steps=4 \
    --gradient_checkpointing \
    --max_train_steps=10000 \
    --validation_epochs=1 \
    --distill_level="sd_tiny"\
    --prepare_unet="True"\
    --use_peft\
    --use_8bit_adam\
    --mixed_precision="bf16"\
    --output_weight=0.5\
    --feature_weight=0.5\
    --learning_rate=1e-05 \
    --max_grad_norm=1 \
    --lr_scheduler="constant"\
    --lr_warmup_steps=0 \
    --report_to="wandb" \
    --lora_r=4 \
    --lora_alpha=32 \
    --push_to_hub \
    --hub_model_id="sd-laion-art"\
    --output_dir="sd-laion-art"\

03/27/2024 14:28:20 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: bf16

{'rescale_betas_zero_snr', 'variance_type', 'timestep_spacing'} was not found in config. Values will be initialized to default values.
{'force_upcast', 'latents_mean', 'latents_std'} was not found in config. Values will be initialized to default values.
{'addition_time_embed_dim', 'encoder_hid_dim_type', 'dropout', 'attention_type', 'transformer_layers_per_block', 'num_attention_heads', 'reverse_transformer_layers_per_block'} was not found in config. Values will be initialized to default values.
{'addition_time_embed_dim', 'encoder_hid_dim_type', 'dropout', 'attention_type', 'transformer_layers_per_block', 'num_attention_heads', 'reverse_transformer_layers_per_block'} was not found in config. Values will be initialized to default values.
Loading dataset shards: 100%|█████████████████| 23/23 [00:00<00:00, 9065.78it/s]
wand

In [24]:
import math

max_train_steps = 100
len_train_dataloader = 100
num_train_epochs = 5
gradient_accumulation_steps = 4

# Scheduler and math around the number of training steps.
overrode_max_train_steps = False
num_update_steps_per_epoch = math.ceil(len_train_dataloader / gradient_accumulation_steps)
if max_train_steps is None:
    max_train_steps = num_train_epochs * num_update_steps_per_epoch
    overrode_max_train_steps = True

# We need to recalculate our total training steps as the size of the training dataloader may have changed.
num_update_steps_per_epoch = math.ceil(len_train_dataloader / gradient_accumulation_steps)
if overrode_max_train_steps:
    max_train_steps = num_train_epochs * num_update_steps_per_epoch

# Afterwards we recalculate our number of training epochs
num_train_epochs = math.ceil(max_train_steps / num_update_steps_per_epoch)

num_train_epochs,max_train_steps, num_update_steps_per_epoch

(4, 100, 25)

In [ ]:
import torch

print(torch.cuda.is_available())

In [ ]:
#srun --partition biggpunodes --nodelist gpunode1 -c 4 --pty --gres=gpu:1 --mem=48G  accelerate launch --mixed_precision="fp16"  lora_distill_training.py --pretrained_model_name_or_path="SG161222/Realistic_Vision_V4.0" --dataset_name="fantasyfish/laion-art" --caption_column=text --resolution=512 --center_crop --random_flip --train_batch_size=1 --num_train_epochs=6 --gradient_accumulation_steps=4 --gradient_checkpointing --max_train_steps=5000 --distill_level=sd_tiny --prepare_unet=True --use_peft --output_weight=0.5 --feature_weight=0.5 --learning_rate=1e-05 --max_grad_norm=1 --lr_scheduler=constant --lr_warmup_steps=0 --validation_prompts= ["A man in a suit"]  --report_to=wandb --lora_r=4 --lora_alpha=32 --push_to_hub --hub_model_id=laion-art --output_dir=sd-laion-art